In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

from citipy import citipy
from config import yelp_key
from config import g_key

In [7]:
rest_df = pd.read_csv("yelp_csv/yelp_csv.csv")
rest_df

,Unnamed: 0,name,category,category_title,rating,price,review_count,address,latitude,longitude,zip code,restaurant_id
0,0,Tortas Frontera,mexican,Mexican,4.0,$$,1395,"['Terminal 3 - Gate K4', ""10000 W O'Hare Ave"",...",41.975438,-87.900137,60666,1zZvM7Zy__Oh533E4bt6_A
1,1,Farmer's Fridge,salad,Salad,4.5,$,62,"['10000 Ohare Way', 'Terminal 3 Gate K6', 'Chi...",41.975234,-87.898719,60666,4xQv1b1IFRsAWZu7yVLFrQ
2,2,Big Bowl,chinese,Chinese,2.0,$$,36,"[""10000 West O'hare Ave"", 'Terminal 5; Gate M7...",41.975103,-87.891350,60666,6VZwwWxBmxMsKd10AIOjxg
3,3,Wow Bao,chinese,Chinese,2.0,$,86,"[""10000 W O'Hare Ave"", 'Terminal 5', 'Chicago,...",41.974346,-87.891078,60666,-kZAA24258sB8CxX_PjQZg
4,4,Hub 51,desserts,Desserts,2.5,$$,62,"[""10000 W O'Hare Ave"", 'Terminal 5', 'Chicago,...",41.974317,-87.890766,60666,6ETGudXoZ9BYXssyiwjArA
...,...,...,...,...,...,...,...,...,...,...,...,...
29908,29908,Maharaj Indian Grill,indpak,Indian,4.5,NaN,26,"['333 S State St', '8 E Van Buren St Unit C13'...",41.877540,-87.627210,60604,RnR_zPbkVcG9G52hgYov_g
29909,29909,Bibibop Asian Grill,asianfusion,Asian Fusion,4.5,$,144,"['24 E Jackson Blvd', 'Chicago, IL 60604']",41.878352,-87.626630,60604,_YicSGcaC7aYqVOYn6iUdQ
29910,29910,STK Steakhouse,steak,Steakhouses,4.0,$$$,1180,"['9 W Kinzie St', 'Chicago, IL 60654']",41.889026,-87.628654,60654,WLcAdMilB7fDFIZLa2vI_w
29911,29911,Lao Sze Chuan,szechuan,Szechuan,3.5,$$,945,"['520 N Michigan Ave', 'Ste 420', 'Chicago, IL...",41.891441,-87.624319,60611,Td-uLb5BUifr7Su-eKKqyw


In [12]:
updated_df = rest_df.set_index("name")
updated_df

,Unnamed: 0,category,category_title,rating,price,review_count,address,latitude,longitude,zip code,restaurant_id
name,,,,,,,,,,,
Tortas Frontera,0,mexican,Mexican,4.0,$$,1395,"['Terminal 3 - Gate K4', ""10000 W O'Hare Ave"",...",41.975438,-87.900137,60666,1zZvM7Zy__Oh533E4bt6_A
Farmer's Fridge,1,salad,Salad,4.5,$,62,"['10000 Ohare Way', 'Terminal 3 Gate K6', 'Chi...",41.975234,-87.898719,60666,4xQv1b1IFRsAWZu7yVLFrQ
Big Bowl,2,chinese,Chinese,2.0,$$,36,"[""10000 West O'hare Ave"", 'Terminal 5; Gate M7...",41.975103,-87.891350,60666,6VZwwWxBmxMsKd10AIOjxg
Wow Bao,3,chinese,Chinese,2.0,$,86,"[""10000 W O'Hare Ave"", 'Terminal 5', 'Chicago,...",41.974346,-87.891078,60666,-kZAA24258sB8CxX_PjQZg
Hub 51,4,desserts,Desserts,2.5,$$,62,"[""10000 W O'Hare Ave"", 'Terminal 5', 'Chicago,...",41.974317,-87.890766,60666,6ETGudXoZ9BYXssyiwjArA
...,...,...,...,...,...,...,...,...,...,...,...
Maharaj Indian Grill,29908,indpak,Indian,4.5,NaN,26,"['333 S State St', '8 E Van Buren St Unit C13'...",41.877540,-87.627210,60604,RnR_zPbkVcG9G52hgYov_g
Bibibop Asian Grill,29909,asianfusion,Asian Fusion,4.5,$,144,"['24 E Jackson Blvd', 'Chicago, IL 60604']",41.878352,-87.626630,60604,_YicSGcaC7aYqVOYn6iUdQ
STK Steakhouse,29910,steak,Steakhouses,4.0,$$$,1180,"['9 W Kinzie St', 'Chicago, IL 60654']",41.889026,-87.628654,60654,WLcAdMilB7fDFIZLa2vI_w


In [20]:
updated_df = updated_df.dropna()


AttributeError: 'function' object has no attribute 'dropna'